In [14]:
import pandas as pd
# Load the dataset
df = pd.read_csv('triton_rezultati.csv')

In [15]:
for i in range(8):
    df[f'Avg_Utililisation {i}'] = df['Avg GPU Utilization'].str.extract(rf'GPU-{i}:(\\\\d*\\\\.?\\\\d*)').astype(float)
    df[f'Avg_Power {i}'] = df['Avg GPU Power Usage'].str.extract(rf'GPU-{i}:(\\\\d*\\\\.?\\\\d*)').astype(float)
    df[f'Max_Memory {i}'] = df['Max GPU Memory Usage'].str.extract(rf'GPU-{i}:(\\\\d*\\\\.?\\\\d*)').astype(float)
    df[f'Total_Memory {i}'] = df['Total GPU Memory'].str.extract(rf'GPU-{i}:(\\\\d*\\\\.?\\\\d*)').astype(float)
df.drop(columns=['Avg GPU Utilization'], inplace=True)
df.drop(columns=['Avg GPU Power Usage'], inplace=True)
df.drop(columns=['Max GPU Memory Usage'], inplace=True)
df.drop(columns=['Total GPU Memory'], inplace=True)

In [16]:
### concatenate test, batch_size, floating_point, kv_cache in one column
df['test'] = (df['test'].astype(str) + '_b' + df['batch_size'].astype(str) + '_' + df['floating_point'].astype(str) +
              '_kv' + df['kv_cache'].astype(str))
df.head()

,test,model,batch_size,floating_point,concurrency,kv_cache,Concurrency,Inferences/Second,Client Send,Network+Server Send/Recv,...,Max_Memory 5,Total_Memory 5,Avg_Utililisation 6,Avg_Power 6,Max_Memory 6,Total_Memory 6,Avg_Utililisation 7,Avg_Power 7,Max_Memory 7,Total_Memory 7
0,alpha_b1_fp16_kv0,gemma3,1,fp16,1:24:2,0,1,0.643508,50,719,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,alpha_b2_fp16_kv0,gemma3,2,fp16,1:24:2,0,19,1.185160,54,819,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,alpha_b2_fp16_kv0,gemma3,2,fp16,1:24:2,0,11,1.203680,48,752,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,alpha_b2_fp16_kv0,gemma3,2,fp16,1:24:2,0,5,1.203680,51,815,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,alpha_b2_fp16_kv0,gemma3,2,fp16,1:24:2,0,9,1.203680,54,820,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
# convert latency columns to seconds from microseconds
df_filtered['p50 latency'] = round(df_filtered['p50 latency'] / 1000000, 3)
df_filtered['p90 latency'] = round(df_filtered['p90 latency'] / 1000000, 3)
df_filtered['p95 latency'] = round(df_filtered['p95 latency'] / 1000000, 3)
df_filtered['p99 latency'] = round(df_filtered['p99 latency'] / 1000000, 3)

In [18]:
# Add a combined metric
df_filtered['throughput_latency_ratio'] = df_filtered['Inferences/Second'] / df_filtered['p99 latency']
# For each test and model, get the row with the highest ratio
best_rows = df_filtered.loc[df_filtered.groupby(['test', 'model'])['throughput_latency_ratio'].idxmax()]

In [19]:
# order by highest throughput_latency_ratio
best_rows = best_rows.sort_values(by='throughput_latency_ratio', ascending=False)
# Display the best rows
best_rows.reset_index(drop=True, inplace=True)
best_rows.head(10)

,test,model,batch_size,floating_point,kv_cache,Concurrency,Inferences/Second,p50 latency,p90 latency,p95 latency,p99 latency,response wait,throughput_latency_ratio
0,alpha_b1_fp16_kv0,gemma3,1,fp16,0,1,0.643508,0.0,0.0,0.0,0.0,1551326,inf
1,alpha_b1_fp16_kv0,meta-llama,1,fp16,0,1,0.370364,0.0,0.0,0.0,0.0,2696545,inf
2,zeta_b8_fp16_kv8192,meta-llama,8,fp16,8192,704,13.925600,0.0,0.0,0.0,0.0,381451185,inf
3,gama_b8_bf16_kv128,mistral,8,bf16,128,1,3.012320,0.0,0.0,0.0,0.0,2651951,inf
4,gama_b8_bf16_kv128,meta-llama,8,bf16,128,1,2.938250,0.0,0.0,0.0,0.0,2718577,inf
5,gama_b8_bf16_kv128,gemma3,8,bf16,128,9,4.555510,0.0,0.0,0.0,0.0,15596771,inf
6,gama_b1_bf16_kv128,mistral,1,bf16,128,1,0.385798,0.0,0.0,0.0,0.0,2595651,inf
7,gama_b1_bf16_kv128,meta-llama,1,bf16,128,1,0.374996,0.0,0.0,0.0,0.0,2666796,inf
8,gama_b1_bf16_kv128,gemma3,1,bf16,128,1,0.651227,0.0,0.0,0.0,0.0,1535922,inf
9,epsilon_b8_bf16_kv4096,mistral,8,bf16,4096,1,3.135770,0.0,0.0,0.0,0.0,2549062,inf


In [20]:
# remove Concurrency column
best_rows.drop(columns=['Concurrency'], inplace=True)
# Display the best rows
best_rows.reset_index(drop=True, inplace=True)
best_rows.head(20)

,test,model,batch_size,floating_point,kv_cache,Inferences/Second,p50 latency,p90 latency,p95 latency,p99 latency,response wait,throughput_latency_ratio
0,alpha_b1_fp16_kv0,gemma3,1,fp16,0,0.643508,0.0,0.0,0.0,0.0,1551326,inf
1,alpha_b1_fp16_kv0,meta-llama,1,fp16,0,0.370364,0.0,0.0,0.0,0.0,2696545,inf
2,zeta_b8_fp16_kv8192,meta-llama,8,fp16,8192,13.925600,0.0,0.0,0.0,0.0,381451185,inf
3,gama_b8_bf16_kv128,mistral,8,bf16,128,3.012320,0.0,0.0,0.0,0.0,2651951,inf
4,gama_b8_bf16_kv128,meta-llama,8,bf16,128,2.938250,0.0,0.0,0.0,0.0,2718577,inf
5,gama_b8_bf16_kv128,gemma3,8,bf16,128,4.555510,0.0,0.0,0.0,0.0,15596771,inf
6,gama_b1_bf16_kv128,mistral,1,bf16,128,0.385798,0.0,0.0,0.0,0.0,2595651,inf
7,gama_b1_bf16_kv128,meta-llama,1,bf16,128,0.374996,0.0,0.0,0.0,0.0,2666796,inf
8,gama_b1_bf16_kv128,gemma3,1,bf16,128,0.651227,0.0,0.0,0.0,0.0,1535922,inf
9,epsilon_b8_bf16_kv4096,mistral,8,bf16,4096,3.135770,0.0,0.0,0.0,0.0,2549062,inf
